<a href="https://colab.research.google.com/github/madhukumarap/-chat_and_notification_feature_codepth.-/blob/madhu/Copy_of_deepfake_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
data_csv='/kaggle/input/dataset/dataset.csv'
data1_dir = '/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'
data2_dir = '/kaggle/input/deepfake-and-real-images/Dataset'
data3_dir = '/kaggle/input/hardfakevsrealfaces'
data4_dir = '/kaggle/input/real-and-fake-face-detection/real_and_fake_face'
data5_dir = '/kaggle/input/real-vs-ai-generated-faces-dataset/dataset/dataset'

In [ ]:
import os
import tensorflow as tf
from matplotlib import pyplot as plt
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from keras.models import load_model
from keras.preprocessing import image
from keras.applications import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
import csv

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'real-and-fake-face-detection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F105271%2F250645%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240714%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240714T151952Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9a4265b632c414aa3b96179901b56bb994e6e9adedea394c9b7b978483cb0652dfb2f9ea9ac6578205a3463af637dd1573f8300b1ed387947dbe389fed2634da59fc118dc3833078b931f3531e583ae8b84fd421b733f6863bb20b680e143ed4bec748f575080f281ffe41cd2569b4957ead7bae5eabc524a081199a1ae8ac9782eff570fd970f7cdc65fcad667bd229dad998ceac8f8e004d86b4f003f778a034f6af1b3af4451a03c9159bfad0f3668c92d92852288328b73364cde116db8955ab523d484636f5cf03c49ac56ebb66e700fb336756d3f72468911e9215615028ca2a872abfd8fc54a6f50a9221ac476fc9057d9fa05f501bef57016b36723b,140k-real-and-fake-faces:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F501529%2F939937%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240714%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240714T151952Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5245b2c2a96b7ed8492dfca48f8583715cce31da61d0585f91d5951801e57c3c4fdfa771403fb4b60ba4172eab9c7f92078876f94124e88c9ccf014ac427a9c8a4830915d032595af3994b5606c4a192c516056dd4d28b14bc2a8edd0a9843ac1680f915af51125aef8a3ca0289a37aaaff76c24ba7723b7f64653c2edea0eaf4b58c6f8b6c105b7a7b9b782b21b24fd41be481785332793ed987af1257bb2be036996920fe5e3eaa11a433243ab9942c5ddc47e1c861e499852207c6e2affd929956a94b0161ae7f5e994edf749a89b1a94c95d869bd8a1bbd85aaad9670172d3debcd9c2d1bb1b8517e6a2a6c4384bda6a0a2aeb931e70b4dfeb2800ab9579,deepfake-and-real-images:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1909705%2F3134515%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240714%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240714T151952Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D45a5dd281a5971860da525a5c824a3a27fea42ff2f8948372f334b47a62ffaa8b287ca73efe61b70e94e303332ae1aa8976c4dd2ba0729b10d3b8185aec6053b8c2258447b95f6b7be6518ede77c49af154199c82e1c61981c556d6129e8a155a9db7725d0d76a32e6e4cdf71dbe63fe433a01776fc68326ed96ca1ce8baa5ff4389f11a3c53dd6b37c5cccb105325b72427c0fec7c8d898d36a1f82d8bcd64e6f811d4b6ebeedf5f981d13f1a6ab8b42938821fbaa8a54b313d4ccd39e64ec558389c9904b61e803e0509d276e1d3067976e1a33ee2c9befde0abbd3768e66e639169816011872128a4b6b204334b29e9e0a1cd1abc7375422854bc277f3716,hardfakevsrealfaces:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1913588%2F3142794%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240714%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240714T151953Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da51d0fdfde125a0a4d21f7f0ae1ddbeafcc9cc22ccb45cd96057a3d8b957fdeb328d575d7d79f1ced3f269308ec4b83bec6b40059cdf8aa938d0c34fbc7201d11b8127ed9d43cdc44ec0a67a921ac9dab361fe786fa7253008c0c8baf0eb89de889d4640b99b4bee91ae3746ddde8b4e77531e672de25aa581e40550067140b235bbeb541d96c285013517efda6a68e26e209a0042c898b6d3e1f885f5e30887787eecc6181a3190d6e968564113d78fdae212dc608bcee6963010d614b2c745303e4718ae7ca3c6ec7970ab7dc922ee37fb67edd3da953313d5e8fe6b2b805fd8795143f5ea3a4f9560af6130f3d7135cfb4019269e1c6e6662ebe546867ca5,real-vs-ai-generated-faces-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4270700%2F7353626%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240714%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240714T151953Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D38b560b53f882b9d181885fb23f2b41880802772c3996d1c38fcd0be87ad1dc53531eb4c9a0147f8cc13ab804d904431ba91db45265f9043957910305994e5c6ecb8f619d9748316dd3be0fd532d436d3e484b4330e486d75998dd3faf1b70523197721865270382ac8e8b562f72253d23cb69922842a46db8ae24df8fc47cd70be898e1072a9a7b6a7c7170ab6ed4c27a9be0972ec62c59544856f653b55939a3e9e6fb96ad12dc6a2832c9c9327dbbb94672478b502220072ad678de2edfdfaf3e7cd1c38545d2495b0e719c344979b1a4110cba8007f8f9f0697ee67e852e6d6c08da8adbf6e91817bd71a77802fc0d64dd5dc7a8b9d591fdc27efe5272c5'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
os.listdir(data5_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/real-vs-ai-generated-faces-dataset/dataset/dataset'

In [ ]:
#print(len( os.listdir((os.path.join(data2_dir,'Train','Real')))))
#print(len( os.listdir((os.path.join(data2_dir,'Train','Fake')))))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/deepfake-and-real-images/Dataset/Train/Real'

In [ ]:
img = cv2.imread(os.path.join(data2_dir,'Train','Real','real_58276.jpg'))

In [ ]:
type(img)

NoneType

In [ ]:
# img.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
# data_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Train'
# data = tf.keras.utils.image_dataset_from_directory(data_dir)

# **creating csv file contains the compination of multiple datasets**

In [ ]:

if os.path.exists(data_csv):
    print('exists')
else:

    data_dirs = [ data3_dir,data4_dir, data5_dir]

    # Define the path for the CSV file
    csv_file = 'dataset1.csv'
    data_csv=csv_file

    # Define the class mapping
    class_mapping = {
        'fake': 'Fake',
        'real': 'Real',
        'Fake': 'Fake',
        'Real': 'Real',
        'training_fake': 'Fake',
        'training_real': 'Real',
        '0':'Real',
        '1':'Fake'
    }

    # Open the CSV file in write mode
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['image', 'label'])  # Write the header row

        # Iterate through the directories in the dataset directory
        for data_dir in data_dirs:
            for root, dirs, files in os.walk(data_dir):
                for file in files:
                    if file.endswith('.jpg') or file.endswith('.png'):  # Adjust file extensions as needed
                        # Get the image path
                        image_path = os.path.join(root, file)

                        # Extract the label from the parent folder name
                        label = os.path.basename(root)

                        # Apply the mapping to the label
                        mapped_label = class_mapping.get(label, label)

                        # Write the image path and mapped label to the CSV file
                        writer.writerow([image_path, mapped_label])


exists


In [ ]:
target_size = (150, 150)

In [ ]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

# Load the CSV file into a DataFrame
df = pd.read_csv(data_csv)

# Create an ImageDataGenerator instance with rescaling and other desired preprocessing options
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Split the data into train/validation sets
)

In [ ]:
# Create a data generator from the DataFrame using flow_from_dataframe()
data_generator = datagen.flow_from_dataframe(
    dataframe=df,
    x_col='image',         # Column containing the image paths
    y_col='label',         # Column containing the labels
    target_size=target_size,
    batch_size=32,
    class_mode='binary',  # Use 'categorical' for multi-class classification
    shuffle=True,           # Shuffle the data
    seed=42,                # Set random seed for reproducibility
    subset='training'       # Use 'training' subset for training data
)

In [ ]:
# Create a validation data generator
validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    x_col='image',
    y_col='label',
    target_size=target_size,
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    seed=42,
    subset='validation'  # Use 'validation' subset for validation data
)

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Create a CNN model
model = Sequential()

# Block 1
model.add(Conv2D(32, (3, 3), activation='relu', padding='same',input_shape=(target_size[0],target_size[1], 3)))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 2
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 3
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 4
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))


# Classification block
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))

model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile('adam',loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()


In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)


In [ ]:
# logdir='logs'

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('best_model.h5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             mode='max',
                             verbose=1)


In [ ]:
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
history = model.fit(data_generator,batch_size=32,epochs=25,validation_data=validation_generator,callbacks=[checkpoint])

In [ ]:
model.save('last_model.h5')

In [ ]:
# hist = model.fit(data_generator, epochs=10, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'], color='teal',label='loss')
plt.plot(history.history['val_loss'], color='orange',label='val_loss')
fig.suptitle('Loss',fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal',label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange',label='val_accuracy')
fig.suptitle('Accuracy',fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
# len(test)

In [ ]:
# for batch in test.as_numpy_iterator():
#   X, y =batch
#   yhat = model.predict(X)
#   pre.update_state(y,yhat)
#   re.update_state(y,yhat)
#   acc.update_state(y,yhat)

In [ ]:
# print("Precision:",pre.result().numpy())
# print("Recall:",re.result().numpy())
# print("Accuracy:",acc.result().numpy())

In [ ]:
# import numpy as np
# import tensorflow as tf
# from sklearn.metrics import confusion_matrix, f1_score

# # Assuming you have a test dataset or generator named 'test'
# # and a pre-trained model named 'model'

# pre = tf.keras.metrics.Precision()
# re = tf.keras.metrics.Recall()
# acc = tf.keras.metrics.Accuracy()

# # Initialize variables for collecting true labels and predicted labels
# true_labels = []
# predicted_labels = []

In [ ]:
# import numpy as np
# from sklearn.metrics import confusion_matrix, f1_score

# # Assuming you have a test dataset or generator named 'test'
# # and a pre-trained model named 'model'

# # Set the threshold for classification
# threshold = 0.5

# # Initialize variables for collecting true labels and predicted probabilities
# true_labels = []
# predicted_labels = []

# # Iterate through the test data and calculate metrics
# for batch in test.as_numpy_iterator():
#     X, y = batch
#     yhat = model.predict(X)

#     # Convert predicted probabilities to class labels using the threshold
#     predicted_labels.extend((yhat >= threshold).astype(int))
#     true_labels.extend(y)

# # Convert lists to numpy arrays
# true_labels = np.array(true_labels)
# predicted_labels = np.array(predicted_labels)

# # Calculate confusion matrix
# cm = confusion_matrix(true_labels, predicted_labels)

# # Calculate F1 score
# f1 = f1_score(true_labels, predicted_labels, average='weighted')

# print("Confusion Matrix:")
# print(cm)
# print("F1 Score:", f1)

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import confusion_matrix

# # Assuming you have the confusion matrix 'cm' calculated

# # Define class labels
# class_labels = ['Class 0', 'Class 1']

# # Create a confusion matrix heatmap
# plt.figure(figsize=(8, 6))
# sns.set(font_scale=1.2)
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
#             annot_kws={'size': 14}, xticklabels=class_labels, yticklabels=class_labels)
# plt.xlabel('Predicted Labels')
# plt.ylabel('True Labels')
# plt.title('Confusion Matrix')
# plt.show()

In [ ]:
# model.save('deep_fake_detection.pb')

In [ ]:
from PIL import Image
def predict_image_class(image_path,model,target_size):
    # Load the pre-trained MobileNet model from TensorFlow Hub

    # Load and preprocess the image
    image = Image.open(image_path)
    image = image.resize(target_size)  # Resize image to match the input size expected by the model
    image = np.array(image) / 255.0    # Normalize pixel values to [0, 1]
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    # Make predictions
    predictions = model.predict(image)
    print(predictions)


In [ ]:
img_path='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test/fake/00F8LKY6JC.jpg'
img=cv2.imread(img_path)
type(img)

In [ ]:
image = Image.open(image_path)
image = image.resize(target_size)  # Resize image to match the input size expected by the model
image = np.array(image) / 255.0    # Normalize pixel values to [0, 1]
image =np.expand_dims(image, axis=0)  # Add batch dimension

In [ ]:
image.shape

In [ ]:
prediction=model.predict(image)

In [ ]:
prediction

In [ ]:
# model1=load_model('/kaggle/working/deep_fake_detection.pb')

In [ ]:
# for batch in test.as_numpy_iterator():
#     X, y =batch
#     yhat = model.predict(X)
#     print(yhat)

In [ ]:
# def load_and_preprocess_image(image_path):
# #     img = image.load_img(image_path, target_size=(None,256, 256,3))
#     img = cv2.imread(os.path.join(data_dir,'Train','Real','real_58276.jpg'))
#     img=cv2.resize(img,(256,256))
#     img_array = image.img_to_array(img)  # Convert PIL image to numpy array
#     return img_array

In [ ]:
# index=0
# for batch in test.as_numpy_iterator():
#     index+=1
#     if index==2:
#         break
#     X, y =batch
#     yhat = model.predict(X)